# Hello Feature Class example: using the feature classes to calculate features

This example shows how to use the Radiomics package to directly instantiate the feature classes for feature extraction. 
Note that this is not the intended standard use. For an example on the standard use with feature extractor, see the `helloRadiomics` example.

In [1]:
from __future__ import print_function, unicode_literals, division, absolute_import
import os
import collections
import SimpleITK as sitk
import numpy
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm

## Getting the test case

Testing data is contained in the pyradiomics/data folder, while this file is in the pyradiomics/bin/Notebooks folder. 

The next line of code gets the location of the current path and gets the location of the data as a relative path by going up two folders ("..") and then move into the data folders ("data").

For this to work, the current active directory should be pyradiomics/bin/notebooks, which is the case if this file is run from the pyradiomics/bin/Notebooks folder.

In [2]:
testCase = 'brain1'
dataDir = os.path.join(os.path.abspath(""), "..", "..", "data")
imageName = os.path.join(dataDir, testCase + '_image.nrrd')
maskName = os.path.join(dataDir, testCase + '_label.nrrd')

if not os.path.exists(imageName):
  print('Error: problem finding input image', imageName)
if not os.path.exists(maskName):
  print('Error: problem finding input labelmap', maskName)

In [3]:
image = sitk.ReadImage(imageName)
mask = sitk.ReadImage(maskName)

## Preprocess the image

#### Extraction Settings

In [4]:
kwargs = {}
kwargs['binWidth'] = 25
kwargs['resampledPixelSpacing'] = None
# kwargs['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
kwargs['interpolator'] = 'sitkBSpline'
kwargs['verbose'] = True

#### If enabled, resample the image

In [5]:
# Resample if necessary
if kwargs['interpolator'] != None and kwargs['resampledPixelSpacing'] != None:
  image, mask = imageoperations.resampleImage(image, mask, kwargs['resampledPixelSpacing'], kwargs['interpolator'])

## Calculate features using original image

In [6]:
# Crop the image
# bb is the bounding box, upon which the image and mask are cropped
croppedImage, croppedMask, bb = imageoperations.cropToTumorMask(image, mask)

### Calculate Firstorder features

In [7]:
firstOrderFeatures = firstorder.RadiomicsFirstOrder(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
firstOrderFeatures.enableFeatureByName('Mean', True)
# firstOrderFeatures.enableAllFeatures()

In [8]:
# Print out the docstrings of the enabled features
print('Will calculate the following first order features: ')
for f in firstOrderFeatures.enabledFeatures.keys():
  print(f)
  print(eval('firstOrderFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following first order features: 
Mean

    Calculate the Mean Value for the image array.

    :math:`mean = \frac{1}{N}\displaystyle\sum^{N}_{i=1}{\textbf{X}(i)}`
    


In [9]:
# Calculate the features and print out result
print('Calculating first order features...')
firstOrderFeatures.calculateFeatures()
print('done')

print('Calculated first order features: ')
for (key, val) in firstOrderFeatures.featureValues.items():
  print('  ', key, ':', val)

Calculating first order features...
done
Calculated first order features: 
   Mean : 825.235436307


### Calculate Shape Features

In [10]:
shapeFeatures = shape.RadiomicsShape(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
# shapeFeatures.enableFeatureByName('Volume', True)
shapeFeatures.enableAllFeatures()

In [11]:
# Print out the docstrings of the enabled features
print('Will calculate the following shape features: ')
for f in shapeFeatures.enabledFeatures.keys():
  print(f)
  print(eval('shapeFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following shape features: 
Elongation


    
Compactness1

    Calculate the compactness (1) of the tumor region.

    :math:`compactness\ 1 = \frac{V}{\sqrt{\pi}A^{\frac{2}{3}}}`

    Compactness 1 is a measure of how compact the shape of the tumor is
    relative to a sphere (most compact). It is a dimensionless measure,
    independent of scale and orientation. Compactness 1 is defined as the
    ratio of volume to the :math:`\sqrt{\text{surface area}^3}`. This is a measure of the
    compactness of the shape of the image ROI
    
SurfaceArea

    Calculate the surface area of the tumor region in square millimeters.

    :math:`A = \displaystyle\sum^{N}_{i=1}{\frac{1}{2}|\textbf{a}_i\textbf{b}_i \times \textbf{a}_i\textbf{c}_i|}`

    Where:

    :math:`N` is the number of triangles forming the surface of the volume

    :math:`a_ib_i` and :math:`a_ic_i` are the edges of the :math:`i`\ :sup:`th` triangle formed by points :math:`a_i`,
    :math:`b_i` and :math:`c_i

In [12]:
# Calculate the features and print out result
print('Calculating shape features...')
shapeFeatures.calculateFeatures()
print('done')

print('Calculated shape features: ')
for (key, val) in shapeFeatures.featureValues.items():
  print('  ', key, ':', val)

FAILED: Traceback (most recent call last):
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/base.py", line 73, in calculateFeatures
    self.featureValues[feature] = eval(call)
  File "<string>", line 1, in <module>
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/shape.py", line 221, in getMaximum2DDiameterSliceFeatureValue
    return self._getMaximum2Ddiameter(0)
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/shape.py", line 126, in _getMaximum2Ddiameter
    for i in numpy.unique(a[:, dim]):
IndexError: too many indices for array

FAILED: Traceback (most recent call last):
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/base.py", line 73, in calculateFeatures
    self.featureValues[feature] = eval(call)
  File "<strin

Calculating shape features...
done
Calculated shape features: 
   Elongation : 1.7789885567018646
   Compactness1 : 26.7546787215
   SurfaceArea : 6438.82160378
   Maximum2DDiameterSlice : nan
   Compactness2 : 0.114127701901
   Flatness : 1.2191850589688844
   Maximum3DDiameter : 65.53661458728622
   Roundness : 0.6146906661500379
   Maximum2DDiameterColumn : nan
   Sphericity : 0.485061744222
   Volume : 16412.65869140624
   SurfaceVolumeRatio : 0.392308261863
   Maximum2DDiameterRow : nan
   SphericalDisproportion : 2.06159321347


### Calculate GLCM Features

In [13]:
glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
# glcmFeatures.enableFeatureByName('SumEntropy', True)
glcmFeatures.enableAllFeatures()

calculate GLCM: 100%|██████████| 33/33 [00:00<00:00, 44.78it/s]


In [14]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLCM features: ')
for f in glcmFeatures.enabledFeatures.keys():
  print(f)
  print(eval('glcmFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following GLCM features: 
ClusterTendency

    Using coefficients :math:`\mu_x` and :math:`\mu_y`, calculate and return the mean Cluster Tendency.

    :math:`cluster\ prominence = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{\big(i+j-\mu_x(i)-\mu_y(j)\big)^2p(i,j)}`

    Cluster Tendency is a measure of groupings of voxels with similar gray-level values.
    
Autocorrelation

    Calculate and return the mean Autocorrelation.

    :math:`autocorrelation = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{p(i,j)ij}`

    Autocorrelation is a measure of the magnitude of the
    fineness and coarseness of texture.
    
AverageIntensity

    Return the mean gray level intensity of the :math:`i` distribution.

    :math:`\mu_x = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{p(i,j)i}`

    N.B. As this formula represents the average of the distribution of :math:`i`, it is independent from the
    distribution of :math:`j`. Therefore, on

In [15]:
# Calculate the features and print out result
print('Calculating GLCM features...')
glcmFeatures.calculateFeatures()
print('done')

print('Calculated GLCM features: ')
for (key, val) in glcmFeatures.featureValues.items():
  print('  ', key, ':', val)

Calculating GLCM features...
done
Calculated GLCM features: 
   ClusterTendency : 103.142793792
   Autocorrelation : 292.684050471
   AverageIntensity : 17.1242601309
   SumEntropy : 5.31547876648
   DifferenceAverage : 5.58932678922
   Entropy : 8.79428086119
   Energy : 0.00290880217681
   Homogeneity2 : 0.189156155892
   SumVariance2 : 103.142793792
   Homogeneity1 : 0.276140402104
   MaximumProbability : 0.00792784235012
   ClusterShade : -52.9707943386
   SumAverage : 33.4497492152
   DifferenceVariance : 17.6107741076
   Idn : 0.866370546902
   ClusterProminence : 26251.1709801
   SumSquares : 39.9781084143
   Id : 0.276140402104
   Idm : 0.189156155892
   InverseVariance : 0.188666637795
   Dissimilarity : 5.58932678922
   Contrast : 52.2310659277
   SumVariance : 895.891808819
   DifferenceEntropy : 3.79686113536
   Imc2 : 0.692033706271
   Correlation : 0.335214788202
   Idmn : 0.957796447609
   Imc1 : -0.091940840043


### Calculate GLRLM Features

In [16]:
glrlmFeatures = glrlm.RadiomicsGLRLM(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
# glrlmFeatures.enableFeatureByName('ShortRunEmphasis', True)
glrlmFeatures.enableAllFeatures()

In [17]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLRLM features: ')
for f in glrlmFeatures.enabledFeatures.keys():
  print(f)
  print(eval('glrlmFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following GLRLM features: 
RunLengthNonUniformity

    Calculate and return the mean Run Length Non-Uniformity (RLN) value for all GLRLMs.

    :math:`RLN = \frac{\sum^{N_r}_{j=1}\left(\sum^{N_g}_{i=1}{\textbf{P}(i,j|\theta)}\right)^2}{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\textbf{P}(i,j|\theta)}}`

    Measures the similarity of run lengths throughout the image, with a lower value indicating
    more homogeneity among run lengths in the image.
    
LongRunEmphasis

    Calculate and return the mean Long Run Emphasis (LRE) value for all GLRLMs.

    :math:`LRE = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\textbf{P}(i,j|\theta)j^2}}{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\textbf{P}(i,j|\theta)}}`

    A measure of the distribution of long run lengths, with a greater value indicative
    of longer run lengths and more coarse structural textures.
    
LowGrayLevelRunEmphasis

    Calculate and return the mean Low Gray Level Run Emphasis (LGLRE) value for all GLRLMs.

    :math:`LGLRE 

In [18]:
# Calculate the features and print out result
print('Calculating GLRLM features...')
glrlmFeatures.calculateFeatures()
print('done')

print('Calculated GLRLM features: ')
for (key, val) in glrlmFeatures.featureValues.items():
  print('  ', key, ':', val)

Calculating GLRLM features...
done
Calculated GLRLM features: 
   RunLengthNonUniformity : 3473.88954354
   LongRunEmphasis : 1.22741432468
   LowGrayLevelRunEmphasis : 0.00846567390082
   RunVariance : 0.0849939088453
   GrayLevelVariance : 39.074009627
   RunPercentage : 0.934010152284
   LongRunLowGrayLevelEmphasis : 0.0104694333711
   GrayLevelNonUniformity : 174.640108304
   RunLengthNonUniformityNormalized : 0.894736783551
   HighGrayLevelRunEmphasis : 281.50156957
   LongRunHighGrayLevelEmphasis : 341.908225705
   ShortRunEmphasis : 0.955813827306
   GrayLevelNonUniformityNormalized : 0.0451916226163
   ShortRunHighGrayLevelEmphasis : 269.366654415
   ShortRunLowGrayLevelEmphasis : 0.0080944625119
   RunEntropy : 4.91343459806


## Calculate Features using Laplacian of Gaussian Filter

Calculating features on filtered images is very similar to calculating features on the original image. All filters in PyRadiomics have the same input and output signature, and there is even one for applying no filter. This enables to loop over a list of requested filters and apply them in the same piece of code. It is applied like this in the execute function in feature extractor. The input for the filters is the image, with additional keywords. If no additional keywords are supplied, the filter uses default values where applicable. It returns a [generator object](https://docs.python.org/2/reference/simple_stmts.html?#yield), allowing to define the generators to be applied before the filters functions are actually called.

### Calculate Firstorder on LoG filtered images

In [20]:
logFeatures = {}
sigmaValues = [1.0, 3.0, 5.0]
for logImage, inputImageName, inputKwargs in imageoperations.getLoGImage(image, sigma=sigmaValues, verbose=True):
  logImage, croppedMask, bb = imageoperations.cropToTumorMask(logImage, mask)
  logFirstorderFeatures = firstorder.RadiomicsFirstOrder(logImage, croppedMask, **inputKwargs)
  logFirstorderFeatures.enableAllFeatures()
  logFirstorderFeatures.calculateFeatures()
  logFeatures[inputImageName] = logFirstorderFeatures.featureValues

	Computing LoG with sigma 1
	Computing LoG with sigma 3
	Computing LoG with sigma 5


In [21]:
# Show result
for sigma, features in logFeatures.items():
  for (key, val) in features.items():
    laplacianFeatureName = '%s_%s' % (str(sigma), key)
    print('  ', laplacianFeatureName, ':', val)

   log-sigma-5-0-mm-3D_RootMeanSquared : 1896.44460614
   log-sigma-5-0-mm-3D_Energy : 14878729370.4
   log-sigma-5-0-mm-3D_Uniformity : 0.0902675928609
   log-sigma-5-0-mm-3D_Maximum : 117.414512634
   log-sigma-5-0-mm-3D_Range : 464.759513855
   log-sigma-5-0-mm-3D_Skewness : -0.30549686903
   log-sigma-5-0-mm-3D_Kurtosis : 3.11489160873
   log-sigma-5-0-mm-3D_Median : -99.1174468994
   log-sigma-5-0-mm-3D_Variance : 6483.79391901
   log-sigma-5-0-mm-3D_10Percentile : -211.974316406
   log-sigma-5-0-mm-3D_Entropy : 3.71336391413
   log-sigma-5-0-mm-3D_TotalEnergy : 59028162175.1
   log-sigma-5-0-mm-3D_InterquartileRange : 106.342716217
   log-sigma-5-0-mm-3D_MeanAbsoluteDeviation : 63.4364264458
   log-sigma-5-0-mm-3D_StandardDeviation : 80.5220089107
   log-sigma-5-0-mm-3D_90Percentile : -10.6977561951
   log-sigma-5-0-mm-3D_Minimum : -347.345001221
   log-sigma-5-0-mm-3D_RobustMeanAbsoluteDeviation : 43.2562957783
   log-sigma-5-0-mm-3D_Mean : -105.265625411
   log-sigma-1-0-mm-3D_

## Calculate Features using Wavelet filter

### Calculate Firstorder on filtered images

In [23]:
waveletFeatures = {}
for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image):
  decompositionImage, croppedMask, bb = imageoperations.cropToTumorMask(decompositionImage, mask)
  waveletFirstOrderFeaturs = firstorder.RadiomicsFirstOrder(decompositionImage, croppedMask, **kwargs)
  waveletFirstOrderFeaturs.enableAllFeatures()
  waveletFirstOrderFeaturs.calculateFeatures()

  print('Calculated firstorder features with ', decompositionName)
  waveletFeatures[decompositionName] = waveletFirstOrderFeaturs.featureValues

Calculated firstorder features with  wavelet-HLL
Calculated firstorder features with  wavelet-LHH
Calculated firstorder features with  wavelet-HHH
Calculated firstorder features with  wavelet-LHL
Calculated firstorder features with  wavelet-LLH
Calculated firstorder features with  wavelet-HLH
Calculated firstorder features with  wavelet-HHL
Calculated firstorder features with  wavelet-LLL


In [24]:
# Show result
for decompositionName, features in waveletFeatures.items():
  for (key, val) in features.items():
    waveletFeatureName = '%s_%s' % (str(decompositionName), key)
    print('  ', waveletFeatureName, ':', val)

   wavelet-HLH_RootMeanSquared : 2000.3397095
   wavelet-HLH_Energy : 16553621990.3
   wavelet-HLH_Uniformity : 0.192969183516
   wavelet-HLH_Maximum : 195.252851267
   wavelet-HLH_Range : 449.325547601
   wavelet-HLH_Skewness : -0.109634215846
   wavelet-HLH_Kurtosis : 5.96264006105
   wavelet-HLH_Median : -0.788893809524
   wavelet-HLH_Variance : 1812.89759077
   wavelet-HLH_10Percentile : -48.7973652241
   wavelet-HLH_Entropy : 2.75501833527
   wavelet-HLH_TotalEnergy : 65672938804.3
   wavelet-HLH_InterquartileRange : 44.1166822627
   wavelet-HLH_MeanAbsoluteDeviation : 30.6419982661
   wavelet-HLH_StandardDeviation : 42.5781351255
   wavelet-HLH_90Percentile : 48.8399961041
   wavelet-HLH_Minimum : -254.072696334
   wavelet-HLH_RobustMeanAbsoluteDeviation : 18.8328798407
   wavelet-HLH_Mean : -0.113489262994
   wavelet-HLL_RootMeanSquared : 1995.50607096
   wavelet-HLL_Energy : 16473718010.5
   wavelet-HLL_Uniformity : 0.163587425574
   wavelet-HLL_Maximum : 186.246123128
   wavel